In [1]:
import mlx.core as mx
import mlx.nn as nn
from mlx.data import datasets
import mlx.optimizers as optim

In [2]:
def get_streamed_data(data, batch_size=0, shuffled=True):
    buffer = data.shuffle() if shuffled else data
    stream = buffer.to_stream()
    stream = stream.key_transform("image", lambda x: x.astype("float32"))
    stream = stream.batch(batch_size) if batch_size > 0 else stream
    return stream.prefetch(4, 2)

In [3]:
# Load training set

mnist_train = datasets.load_mnist(train=True)
mnist_test = datasets.load_mnist(train=False)

In [4]:
# Get one sample from the train set
stream = get_streamed_data(mnist_train, batch_size=1, shuffled=False)
batch = next(stream)
X, y = mx.array(batch["image"]), mx.array(batch["label"])
X.shape, y.shape

((1, 28, 28, 1), (1,))

In [5]:
# Create a convolution sequence
conv_seq1 = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2)
)

X_out1 = conv_seq1(X)
print(X_out1.shape)

(1, 13, 13, 1)


In [6]:
# Calculating output dimension of a Conv2d layer
def get_convolution_output_dim(width, stride, kernel_size, padding):
    """
    Assuming that the input image has dimension of width x width (square image).
    We only need the width in this case
    """ 
    return int(((width - kernel_size + (2 * padding)) / stride)) + 1

def get_pooling_layer_out_dim(width, height, stride, kernel_size, padding):
    """
    Assuming that stride and kernel is a square (e.g., 2x2)
    """
    w_out = ((width + 2 * padding - kernel_size) / stride) + 1
    h_out = ((height + 2 * padding - kernel_size) / stride) + 1
    return int(w_out), int(h_out)


In [7]:
# So long the convolution/pooling kernel size, padding, and stride is configurable
# We can calculate each output dimension without passing a sample
input_image_width = X.shape[-2]
conv_padding = 0
conv_stride = 1
conv_kernel_size = 3

pool_padding = 0
pool_stride = 2
pool_kernel_size = 2

conv_seq_1_conv_out_dim = get_convolution_output_dim(input_image_width, conv_stride, conv_kernel_size, conv_padding)
conv_seq_1_pooling_out_dim = get_pooling_layer_out_dim(
    conv_seq_1_conv_out_dim,
    conv_seq_1_conv_out_dim,
    pool_stride,
    pool_kernel_size,
    pool_padding
)

print(conv_seq_1_pooling_out_dim)

conv_seq_2_conv_out_dim = get_convolution_output_dim(
    conv_seq_1_pooling_out_dim[-1],
    conv_stride,
    conv_kernel_size,
    conv_padding
)
conv_seq_2_pooling_out_dim = get_pooling_layer_out_dim(
    conv_seq_2_conv_out_dim,
    conv_seq_2_conv_out_dim,
    pool_stride,
    pool_kernel_size,
    pool_padding
)

print(conv_seq_2_pooling_out_dim)

(13, 13)
(5, 5)


In [8]:
class ConvolutedMNISTModel(nn.Module):
    def __init__(self, input_channel, input_width, output_dims):
        super().__init__()
        conv2d_kernel_size = 3
        conv2d_stride = 1
        conv2d_padding = 0

        pool2d_kernel_size = 2
        pool2d_stride = 2
        pool2d_padding = 0

        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_channel,
                out_channels=input_channel,
                kernel_size=conv2d_kernel_size,
                stride=conv2d_stride,
                padding=conv2d_padding
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool2d_kernel_size, stride=pool2d_stride, padding=pool2d_padding),
        )

        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_channel,
                out_channels=input_channel,
                kernel_size=conv2d_kernel_size,
                stride=conv2d_stride,
                padding=conv2d_padding
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool2d_kernel_size, stride=pool2d_stride, padding=pool2d_padding),
        )

        self.fully_connected = nn.Sequential(
            nn.Linear(input_dims=5*5*input_channel, output_dims=16),
            nn.ReLU(),
            nn.Linear(input_dims=16, output_dims=16),
            nn.ReLU(),
            nn.Linear(input_dims=16, output_dims=output_dims),
        )

    def __call__(self, x):
        x = self.conv_layer1(x)
        # print(f"conv_layer1.out_shape -> {x.shape}")
        x = self.conv_layer2(x)
        # print(f"conv_layer2.out_shape -> {x.shape}")
        x = mx.flatten(x, start_axis=1) # ignore the batch shape (axis=0)
        # print(f"mx.flatten.out_shape -> {x.shape}")
        x = self.fully_connected(x)
        return x

In [9]:
NUM_CLASSES = 10 # number of classes
INPUT_WIDTH = 28 # image width -> assuming a square image
INPUT_CHANNEL = 1 # 1 color  channel

model = ConvolutedMNISTModel(
    input_channel=INPUT_CHANNEL,
    input_width=INPUT_WIDTH,
    output_dims=NUM_CLASSES
)

mx.eval(model)
model

ConvolutedMNISTModel(
  (conv_layer1): Sequential(
    (layers.0): Conv2d(1, 1, kernel_size=(3,), stride=(1, 1), padding=(0, 0), dilation=1, bias=True)
    (layers.1): ReLU()
    (layers.2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  )
  (conv_layer2): Sequential(
    (layers.0): Conv2d(1, 1, kernel_size=(3,), stride=(1, 1), padding=(0, 0), dilation=1, bias=True)
    (layers.1): ReLU()
    (layers.2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  )
  (fully_connected): Sequential(
    (layers.0): Linear(input_dims=25, output_dims=16, bias=True)
    (layers.1): ReLU()
    (layers.2): Linear(input_dims=16, output_dims=16, bias=True)
    (layers.3): ReLU()
    (layers.4): Linear(input_dims=16, output_dims=10, bias=True)
  )
)

In [10]:
def loss_fn(model, X, y):
    logits = model(X)
    return nn.losses.cross_entropy(logits, y, reduction="mean")

def eval_fn(model, X, y):
    logits = model(X)
    pred = nn.softmax(logits)
    return mx.mean(mx.argmax(pred, axis=1) == y)

In [ ]:
# Start training loop

epochs = 50
loss_and_grad_fn = nn.value_and_grad(model, loss_fn)

optimizer = optim.SGD(learning_rate=0.01)
for epoch in range(epochs):
    epoch_loss = 0.0
    train_acc = 0.0
    epoch_counter = 0
    for batch in get_streamed_data(mnist_train, batch_size=256, shuffled=True):
        X, y = batch["image"], batch["label"]
        # Need to convert X and y into mlx.core.array type
        X, y = mx.array(X), mx.array(y)
        # Compute loss and its gradient with respect to the model's trainable parameters
        loss, grad = loss_and_grad_fn(model, X, y)
        # Step the optimizer
        optimizer.update(model, grad)
        # Evaluate computational graph
        mx.eval(model.parameters(), optimizer.state)
        epoch_loss += loss.item()
        epoch_counter += 1
        train_acc += eval_fn(model, X, y).item()
    epoch_loss /= epoch_counter
    train_acc /= epoch_counter

    test_acc_counter = 0.0
    test_acc = 0.0
    for batch in get_streamed_data(mnist_test, batch_size=32, shuffled=False):
        X, y = batch["image"], batch["label"]
        X, y = mx.array(X), mx.array(y)
        acc = eval_fn(model, X, y)
        test_acc += acc.item()
        test_acc_counter += 1
    test_acc /= test_acc_counter
    print(f"Epoch: {epoch} | Train Loss: {epoch_loss}, Train Accuracy: {train_acc} | Test Accuracy: {test_acc}")

Epoch: 0 | Train Loss: 1.9087590671600179, Train Accuracy: 0.2977670656873825 | Test Accuracy: 0.6873003194888179
Epoch: 1 | Train Loss: 0.685671506283131, Train Accuracy: 0.804471409574468 | Test Accuracy: 0.8404552715654952
Epoch: 2 | Train Loss: 0.4817481907124215, Train Accuracy: 0.8681959220703612 | Test Accuracy: 0.8607228434504792
Epoch: 3 | Train Loss: 0.4174379700041832, Train Accuracy: 0.886846188027808 | Test Accuracy: 0.8851837060702875
Epoch: 4 | Train Loss: 0.3890635477735641, Train Accuracy: 0.8958111702127659 | Test Accuracy: 0.8976637380191693
Epoch: 5 | Train Loss: 0.3685422419233525, Train Accuracy: 0.9015403369639782 | Test Accuracy: 0.8973642172523961
Epoch: 6 | Train Loss: 0.3554524838924408, Train Accuracy: 0.9047872340425532 | Test Accuracy: 0.9034544728434505
Epoch: 7 | Train Loss: 0.3467174992916432, Train Accuracy: 0.9068816489361702 | Test Accuracy: 0.9095447284345048
Epoch: 8 | Train Loss: 0.3388098003382378, Train Accuracy: 0.9091866135597229 | Test Accura

In [ ]:
# Testing out with one random sample
test_stream = get_streamed_data(mnist_test, batch_size=1)
test_batch = next(test_stream)
X, y = mx.array(test_batch["image"]), mx.array(test_batch["label"])

# See how the model produce logits
logits = model(X)
print(f"Logits: {logits}")

# See we can compute the softmax from the logits
softmax = nn.softmax(logits)
print(f"Softmax-ed: {softmax}")

# Get predicted label and true label
predicted_label = mx.argmax(softmax, axis=1).item()
confidence_level = mx.max(softmax, axis=1).item()
print(f"Predicted label: {predicted_label}, True label: {y.item()} | Confidence level: {confidence_level}")

In [ ]:
# get precision, recall, and f1-score
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

y_true = []
y_pred = []
for batch in get_streamed_data(mnist_test, batch_size=32, shuffled=False):
    X, y = batch["image"], batch["label"]
    X, y = mx.array(X), mx.array(y)
    logits = model(X)
    prediction = mx.argmax(mx.softmax(logits), axis=1)
    y_true = y_true + y.tolist()
    y_pred = y_pred + prediction.tolist()
    
y_true = np.array(y_true)
y_pred = np.array(y_pred)

precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")
print(f"Precision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(conf_matrix)
display.plot(cmap=plt.cm.Blues)
plt.show()